In [1]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
tf.compat.v1.experimental.output_all_intermediates(True)
from keras.applications import ResNet152
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras import backend as K

import numpy as np
import os
import matplotlib.pyplot as plt
import cv2

Using TensorFlow backend.


In [6]:
model_res = ResNet152(weights="imagenet", include_top=False, pooling='avg')

def load_image(image_path):
    image = load_img(image_path, target_size=(224,224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = imagenet_utils.preprocess_input(image)
    return image

def predict(model, image_path):
    image = load_image(image_path)
    return model.predict(image)

In [7]:
predict(model_res, "imgs/1.png")

array([[0.03061284, 0.18128711, 0.5147286 , ..., 0.5300417 , 0.64169264,
        1.8828745 ]], dtype=float32)